In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,24586
2,Huelva,Confirmados PDIA 14 días,350
3,Huelva,Tasa PDIA 14 días,"68,20351930159596"
4,Huelva,Confirmados PDIA 7 días,116
5,Huelva,Total Confirmados,24808
6,Huelva,Curados,16310
7,Huelva,Fallecidos,304


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  24586.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6550.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 4960 personas en los últimos 7 días 

Un positivo PCR cada 1821 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,24586.0,116.0,350.0,513170.0,22.604595,68.203519,40.0
Huelva-Costa,14995.0,71.0,207.0,289548.0,24.520978,71.490737,24.0
Condado-Campiña,6872.0,42.0,131.0,156231.0,26.883269,83.850196,15.0
Almonte,863.0,15.0,37.0,24507.0,61.207002,150.977272,9.0
Cartaya,946.0,11.0,31.0,20083.0,54.772693,154.359408,7.0
Lepe,1994.0,8.0,33.0,27880.0,28.694405,118.364419,5.0
Ayamonte,1193.0,7.0,14.0,21104.0,33.169067,66.338135,4.0
Huelva (capital),6550.0,29.0,79.0,143837.0,20.161711,54.923281,4.0
Puebla de Guzmán,87.0,6.0,7.0,3092.0,194.049159,226.390686,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villablanca,176.0,2.0,8.0,2885.0,69.324090,277.296360,2.0
Puebla de Guzmán,87.0,6.0,7.0,3092.0,194.049159,226.390686,3.0
Lucena del Puerto,152.0,5.0,6.0,3261.0,153.327200,183.992640,1.0
Cartaya,946.0,11.0,31.0,20083.0,54.772693,154.359408,7.0
Almonte,863.0,15.0,37.0,24507.0,61.207002,150.977272,9.0
Palma del Condado (La),976.0,6.0,14.0,10801.0,55.550412,129.617628,2.0
Lepe,1994.0,8.0,33.0,27880.0,28.694405,118.364419,5.0
Moguer,827.0,8.0,24.0,21867.0,36.584808,109.754424,2.0
Bollullos Par del Condado,608.0,1.0,15.0,14387.0,6.950719,104.260791,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Bollullos Par del Condado,608.0,1.0,15.0,14387.0,6.950719,104.260791,0.0,0.066667
Rociana del Condado,310.0,1.0,7.0,7939.0,12.596045,88.172314,1.0,0.142857
Sierra de Huelva-Andévalo Central,2298.0,1.0,6.0,67391.0,1.483878,8.903266,0.0,0.166667
Trigueros,228.0,1.0,5.0,7862.0,12.719410,63.597049,0.0,0.200000
Lepe,1994.0,8.0,33.0,27880.0,28.694405,118.364419,5.0,0.242424
Punta Umbría,657.0,2.0,8.0,15355.0,13.025073,52.100293,-1.0,0.250000
Villablanca,176.0,2.0,8.0,2885.0,69.324090,277.296360,2.0,0.250000
Aljaraque,924.0,3.0,11.0,21474.0,13.970383,51.224737,1.0,0.272727
Palos de la Frontera,449.0,2.0,7.0,11742.0,17.032873,59.615057,0.0,0.285714
